# Sesión 3.2. Teoría - Redes Neuronales Recurrentes
Curso 2022-23

Profesor: [Jorge Calvo Zaragoza](mailto:jcalvo@dlsi.ua.es)

## Resumen
En esta sesión:
  * Introducimos las redes neuronales recurrentes (RNN).
  * Veremos el uso de RNN en Keras.

## Introducción
  
* Determinados problemas no se pueden resolver usando solamente la información del instante actual.

* Por ejemplo, qué está sucediendo en una película viendo solamente una imagen, el rol o significado de una palabra sin consultar su contexto, o más sencillo, el siguiente valor de una serie basándonos solamente en el valor anterior.

.


![Serie temporal](http://www.dlsi.ua.es/~jgallego/deepraltamira/serie_temporal.png)

.

* La redes neuronales tradicionales no pueden hacer esto: sólo condicionan su salida a la entrada.
  * Es decir, para una misma entrada, darán siempre la misma salida.

* Para solucionarlo surgieron las Redes Neuronales Recurrentes (RNN).

* La primera red neuronal recurrente fue propuesta por [Jordan en 1986](https://eric.ed.gov/?id=ED276754).
* Posteriormente, en [1997, Hochreiter 	y Schmidhuber](http://www.bioinf.jku.at/publications/older/2604.pdf) propusieron las neuronas tipo LSTM, consideradas clave en el despegue definitivo del Deep Learning para procesamiento de secuencias (y, concretamente, procesamiento de lenguaje natural).

.

![Deep Learning Timeline](http://www.dlsi.ua.es/~jgallego/deepraltamira/deep_learning_timeline_rnn.png)




## Neuronas recurrentes

* Las neuronas recurrentes son similares a las neuronas normales pero con **bucles recurrentes** que les permiten mantener un **estado** en el tiempo.

* Esta neurona predice su salida ($h_t$) a partir de la entrada en ese instante ($x_t$) y también a partir de la salida del estado anterior ($h_{t-1}$).

--

![Neurona recurrente](http://www.dlsi.ua.es/~jgallego/deepraltamira/rnn2.png)

--

* El estado interior de la neurona va cambiando o evolucionando en función de lo que ha visto hasta ese momento, transmitiendo la información que hace falta para el problema de aprendizaje en cuestión.

* También podemos pensar en las neuronas recurrentes de forma "desplegada", en la que tendríamos tantas neuronas como elementos de la secuencia y cada una le pasa su estado a la siguiente.

--

![RNN unrolled](http://www.dlsi.ua.es/~jgallego/deepraltamira/rnn_unrolled1.png)






## Formulaciones


* Con estas neuronas se pueden formular problemas de distinta naturaleza.

.

![Esquemas con RNN](http://www.dlsi.ua.es/~jgallego/deepraltamira/rnn_diagrams.png)

.
* Clasificándolos según su entrada y su salida tendríamos:
    * **Uno a uno:** clasificación o regresión convencional en el cual a una entrada le corresponde una salida.
    * **Muchos a uno:** clasificación de secuencias, en las cuales queremos asignar una categoria a una secuencia. Ejemplo: análisis de sentimiento de un texto.
    * **Uno a muchos:** tareas en las cuáles se quiere obtener una secuencia a partir de una única entrada. Ejemplo: descripción automática de imagen.  
    * **Muchos a muchos (desacopladas o no):** tareas en las cuales tanto la entrada como la salida son secuencias. Se puede plantear de diferente forma dependiendo de si las secuencias son desacopladas (traducción automática) o si hay una fuerte relación entre cada elemento de la entrada y la salida (etiquetado gramatical).

  
 * En las sesiones anteriores hemos visto la formulación "Uno a uno".

 * A continuación vamos a ver la formulación "Muchos a uno".



## Problema de las dependencias a largo plazo

* En algunas tareas solo es necesario utilizar la información más reciente para calcular la siguiente predicción.

  Es el caso de las series temporales solo necesitamos consultar los $n$ elementos anteriores. O para aprender a predecir la siguiente palabra en una frase sencilla, como por ejemplo "*Las nubes están en el  [ ?  ]*"
  
.

![short term depdencies](http://www.dlsi.ua.es/~jgallego/deepraltamira/rnn_shorttermdepdencies1.png)

.


* Pero a veces sí que es necesario consultar más contexto para obtener la siguiente predicción.

  Por ejemplo, para predecir la siguiente palabra en la frase "*Nací en Francia pero a los 12 años me vine a España, ... por eso es que hablo tan bien el [ ? ]*".
  
  El contexto reciente sugiere que la siguiente palabra es un idioma pero para saber el idioma correcto tenemos que utilizar la información del principio de la frase.

.

![long term dependencies](http://www.dlsi.ua.es/~jgallego/deepraltamira/rnn_longtermdependencies1.png)

.

* Este es un problema para las RNN, dado que según aumenta la distancia hasta la información a utilizar, se vuelve más difícil aprender a conectar la información.

* Cuando la información hay que propagarla *muy atrás en el tiempo*, el gradiente de las funciones de activación tiende a 0. Esto se conoce como *vanishing gradient* y constituye uno de los problemás típicos de las redes neuronales.

* Para lidiar con esta cuestión surgieron las neuronas tipo LSTM.


## Neuronas LSTM (Long Short-Term Memory)

* Su diseño permite *recordar* información tanto a corto como a largo plazo.

* La principal diferencia con las neuronas recurrentes es que incluyen (internamente) una celda o bucle de memoria.

* Gracias a esto reducen el efecto del "*vanishing gradient*" durante el entrenamiento.

.

![Neurona LSTM](http://www.dlsi.ua.es/~jgallego/deepraltamira/lstm.png)

.

* Las LSTM incluyen una serie de puertas que permiten controlar la información que entra y sale de la celda de memoria:

 * **Puerta de entrada:** controla los valores de entrada que se van a utilizar para actualizar el estado de la memoria.
 * **Puerta de salida:** controla los valores a devolver a partir de la entrada y del contenido de la memoria.
 * **Puerta de olvido:** controla el mantenimiento del contenido de la memoria.

* Existe una versión simplificada de estas neuronas llamadas GRU (Gated Recurrent Units), sin memoria interna y con menos puertas, y por lo tanto con menos parámetros a aprender.



## RNN con Keras

* Con Keras podemos crear RNN usando la clase `LSTM`.

* La clase [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/) permite añadir capas con neuronas de este tipo.

  Como parámetro recibe el número de neuronas a, por ejemplo "`LSTM(32)`".
  
  Existen otros parámetros importantes que iremos viendo durante el curso.








### Formato de los datos para Keras

* Para entrenar y validar las redes recurrentes tenemos que formatear los datos en secuencias.

* Para la arquitectura "muchos a uno", cada muestra o ejemplo de entrenamiento constará de una secuencia y de una etiqueta, que se calcula en la salida del último estado de la red recurrente.

  Por ejemplo:

\begin{equation}
\begin{matrix}
X & & Y \\
\begin{bmatrix}
  \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix} \\
  \begin{bmatrix} 2 \\ 3 \\ 4 \end{bmatrix} \\
  \begin{bmatrix} 3 \\ 4 \\ 5 \end{bmatrix}  \\
  \begin{bmatrix} 4 \\ 5 \\ 1 \end{bmatrix}
\end{bmatrix}
& &
\begin{bmatrix}
\\ \\ 4 \\
\\ \\ 5 \\
\\ \\ 1 \\
\\ \\ 2 \\
\end{bmatrix}
\end{matrix}
\end{equation}


* Las dimensiones de estas matrices serían:

          X = (4, 3)
          Y = (4)


* Pero para implementar este tipo de redes en Keras (RNN para secuencias temporales) se espera recibir como entrada un vector de **3 dimensiones** y como salida uno de 2. Recordad que la última dimension hace referencia al tamaño de las características, que hay que añadir incluso aunque sea **1** (como ocurre con los canales de una imagen en escala de grises).

* Por tanto, para las X tendríamos:
 * En la primera dimensión el número de ejemplos.
 * En la segunda dimensión la longitud de la secuencia.
 * Y en la tercera dimensión las características, que en este caso es un único valor numérico.
* Para las Y solo se necesitan 2 dimensiones: la primera con el número de ejemplos y la segunda para el valor a predecir.

\begin{equation}
\begin{matrix}
X & & Y \\
\begin{bmatrix}
  \begin{bmatrix} [1] \\ [2] \\ [3] \end{bmatrix} \\
  \begin{bmatrix} [2] \\ [3] \\ [4] \end{bmatrix} \\
  \begin{bmatrix} [3] \\ [4] \\ [5] \end{bmatrix}  \\
  \begin{bmatrix} [4] \\ [5] \\ [1] \end{bmatrix}
\end{bmatrix}
& &
\begin{bmatrix}
\\ \\ [4] \\
\\ \\ [5] \\
\\ \\ [1] \\
\\ \\ [2] \\
\end{bmatrix}
\end{matrix}
\end{equation}



* En este caso las dimensiones de estas matrices serían:

          X = (4, 3, 1)
          Y = (4, 1)


* Y en el caso de que utilicemos la codificación "one-hot" o categórica, tendríamos:


\begin{equation}
\begin{matrix}
X & & Y \\
\begin{bmatrix}
  \begin{bmatrix}  
    \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 1 & 0 & 0 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 1 & 0 & 0 \end{bmatrix}
  \end{bmatrix}
  \\
  \begin{bmatrix}
    \begin{bmatrix} 0 & 1 & 0 & 0 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 1 & 0 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 0 & 1 & 0 \end{bmatrix}
  \end{bmatrix}
  \\
  \begin{bmatrix}
    \begin{bmatrix} 0 & 0 & 1 & 0 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 0 & 1 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 0 & 0 & 1 \end{bmatrix}
  \end{bmatrix}  
  \\
  \begin{bmatrix}
    \begin{bmatrix} 0 & 0 & 0 & 1 & 0 \end{bmatrix}
    \\
    \begin{bmatrix} 0 & 0 & 0 & 0 & 1 \end{bmatrix}
    \\
    \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix}
  \end{bmatrix}
\end{bmatrix}
& &
\begin{bmatrix}
\\ \\ \begin{bmatrix} 0 & 0 & 0 & 1 & 0 \end{bmatrix} \\
\\ \\ \begin{bmatrix} 0 & 0 & 0 & 0 & 1 \end{bmatrix} \\
\\ \\ \begin{bmatrix} 1 & 0 & 0 & 0 & 0 \end{bmatrix} \\
\\ \\ \begin{bmatrix} 0 & 1 & 0 & 0 & 0 \end{bmatrix} \\
\end{bmatrix}
\end{matrix}
\end{equation}

* En este caso las dimensiones serían:

          X = (4, 3, 5)
          Y = (4, 5)



### Ejemplo de RNN

Vamos a implementar nuestra primera red recurrente para ajustarse con los datos del ejemplo anterior.

In [1]:
%%capture --no-stdout

import tensorflow as tf
import numpy as np

# ---------------------
# Datos del ejemplo
#
print()
print('Creamos datos...')

X = np.asarray([
     [ [1,0,0,0,0],
       [0,1,0,0,0],
       [0,0,1,0,0]
     ],
     [ [0,1,0,0,0],
       [0,0,1,0,0],
       [0,0,0,1,0]
     ],
     [ [0,0,1,0,0],
       [0,0,0,1,0],
       [0,0,0,0,1]
     ],
     [ [0,0,0,1,0],
       [0,0,0,0,1],
       [1,0,0,0,0]
     ],
    ])

Y = np.asarray([
      [0,0,0,1,0],
      [0,0,0,0,1],
      [1,0,0,0,0],
      [0,1,0,0,0]
    ])


print('X: {}'.format(X.shape))
print('Y: {}'.format(Y.shape))

# ---------------------
# Creamos red
#
print()
print('Creamos red...')

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer((3, 5))) # Longitud de las secuencias , características
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Dense(5))                    # Categorías del ejemplo
model.add(tf.keras.layers.Activation('softmax'))       # Clasificación
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop')              # RMSProp es típico de RNN
model.summary()

# ---------------------
# Entrenamiento
#
print()
print('Entrenamiento...')

model.fit(X,Y,epochs=100,verbose=0)

# ---------------------
# Datos  del ejemplo
#
print()
print('Predicción...')

prediccion = model.predict(X)
categorias = np.argmax(prediccion,axis=1)+1

for idx_s, secuencia in enumerate(X):
  print()
  print('Secuencia:')
  print(secuencia)
  print('Prediccion: {}'.format(categorias[idx_s]))


Creamos datos...
X: (4, 3, 5)
Y: (4, 5)

Creamos red...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4864      
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                                 
Total params: 5,029
Trainable params: 5,029
Non-trainable params: 0
_________________________________________________________________

Entrenamiento...

Predicción...
1/1 [==============================] - 1s 1s/step

Secuencia:
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]]
Prediccion: 4

Secuencia:
[[0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]]
Prediccion: 5

Secuencia:
[[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 

&nbsp;

&nbsp;


---

 Vamos a practicar &#10158; [Predicción de los valores de bolsa](https://colab.research.google.com/drive/1-4fHNwYT5UTTQZ5qhN3hpTVei7IwkYEd?usp=sharing)





---

### Otras características de las redes recurrentes

#### Predicción en cada instante temporal

Ya hemos visto que las redes recurrentes son adecuadas para tratar con problemas de naturaleza secuencial.

A diferencia de lo visto hasta ahora, también podemos obtener una salida para cada instante temporal mediante un parámetro de la capa recurrente:

* **return_sequences**: cuando está desactivado (por defecto), sólo devuelve la salida del último instante de tiempo; cuando está activado, devuelve la salida de todos los instantes.

Atención a la salida de la LSTM según se active o no este parámetro:



In [3]:
# La entrada a una red recurrente tiene dos dimensiones:
# - El número de instantes de tiempo (T)
# - Número de características de entrada (F)
import tensorflow as tf

T = 100
F = 2
capa_entrada = tf.keras.layers.Input(shape=(T, F))

x0 = tf.keras.layers.LSTM(64)(capa_entrada)
x1 = tf.keras.layers.LSTM(64,return_sequences=True)(capa_entrada)

print('LSTM: ' + str(x0))
print('LSTM + return_sequences: ' + str(x1))
print()

LSTM: KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='lstm_3/PartitionedCall:0', description="created by layer 'lstm_3'")
LSTM + return_sequences: KerasTensor(type_spec=TensorSpec(shape=(None, 100, 64), dtype=tf.float32, name=None), name='lstm_4/PartitionedCall:1', description="created by layer 'lstm_4'")



Esto permite, por lo tanto, construir una red neuronal para predecir un valor en cada instante temporal.

En el este ejemplo entrenamos una red neuronal que nos diga, para cada instante de una secuencia temporal, si el número de entrada es igual al anterior. En este caso, los datos serían:

\begin{equation}
\begin{matrix}
X & & Y \\
\begin{bmatrix}
  \begin{bmatrix} 1 \\ 0 \\ 1 \\ 1 \end{bmatrix} \\
  \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix} \\
  \begin{bmatrix} 1 \\ 1 \\ 0 \\ 0 \end{bmatrix}  \\
  \begin{bmatrix} 1 \\ 1 \\ 1 \\ 0 \end{bmatrix}  \\
  \begin{bmatrix} 0 \\ 1 \\ 1 \\ 0 \end{bmatrix}
\end{bmatrix}
& &
\begin{bmatrix}
  \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix} \\
  \begin{bmatrix} 0 \\ 1 \\ 1 \\ 0 \end{bmatrix} \\
  \begin{bmatrix} 0 \\ 1 \\ 0 \\ 1 \end{bmatrix}  \\
  \begin{bmatrix} 0 \\ 1 \\ 1 \\ 0 \end{bmatrix}  \\
  \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix}
\end{bmatrix}
\end{matrix}
\end{equation}


* Las dimensiones de estas matrices serían:

          X = (5, 4)
          Y = (5, 4)

* Y si preparamos las matrices para los lotes de Keras, habría que añadir la dimensión extra de las características:

          X = (5, 4, 1)
          Y = (5, 4, 1)

En esta ocasión, como no tenemos entrada categórica, el propio valor ($0$ o $1$) es suficiente para representar la entrada.

In [4]:

import numpy as np


# ---------------------
# Datos del ejemplo
#

print()
print('Creamos datos...')

X = np.asarray([
       [1,0,1,1],
       [0,0,0,1],
       [1,1,0,0],
       [1,1,1,0],
       [0,1,1,0]
    ])

Y = np.asarray([
       [0,0,0,1],
       [0,1,1,0],
       [0,1,0,1],
       [0,1,1,0],
       [0,0,1,0]
    ])

X = np.expand_dims(X,axis=-1) # Añadimos una dimension "ficticia" al final
Y = np.expand_dims(Y,axis=-1) # Añadimos una dimension "ficticia" al final


print('X: {}'.format(X.shape))
print('Y: {}'.format(Y.shape))

# ---------------------
# Creamos red
#
print()
print('Creamos red...')

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(4, 1))) # Longitud de las secuencias X características
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))                    # Clasificacion binaria
model.compile(loss='binary_crossentropy', optimizer='rmsprop')              # RMSProp es típico de RNN
model.summary()

# ---------------------
# Entrenamiento
#
print()
print('Entrenamiento...')

model.fit(X,Y,epochs=500,verbose=0)  # Suficientes épocas para memorizar

# ---------------------
# Datos  del ejemplo
#
print()
print('Predicción...')

prediccion = model.predict(X)

for idx_s, secuencia in enumerate(X):
  print()
  print('Secuencia: {}'.format(np.squeeze(secuencia)))
  print('Prediccion: {}'.format(np.squeeze(prediccion[idx_s])))
  print('Decision: {}'.format( np.squeeze(prediccion[idx_s]) > 0.5 ))
  print('Valor correcto: {}'.format(np.squeeze(Y[idx_s])))


Creamos datos...
X: (5, 4, 1)
Y: (5, 4, 1)

Creamos red...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 4, 64)             16896     
                                                                 
 dense_1 (Dense)             (None, 4, 1)              65        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________

Entrenamiento...

Predicción...
1/1 [==============================] - 0s 462ms/step

Secuencia: [1 0 1 1]
Prediccion: [0.00776608 0.5111109  0.06061182 0.99766654]
Decision: [False  True False  True]
Valor correcto: [0 0 0 1]

Secuencia: [0 0 0 1]
Prediccion: [9.8397555e-03 6.0357147e-01 9.8159659e-01 8.9418102e-04]
Decision: [False  True  True False]
Valor correcto: [0 1 1 0]

Sec

Obviamente, este ejemplo tiene poco sentido puesto que la red neuronal memoriza fácilmente. Sin embargo, permite observar cómo modelar un problema de **muchos a muchos**.

#### Capas recurrentes apiladas

Si queremos apilar varias capas recurrentes consecutivas (como se hace con las CNN) necesitamos hacer uso de *return_sequences* también. De esta forma, la neurona en un instante temporal propaga su salida a la siguiente capa. Esto es independiente de si la ultima capa activa o no *return_sequences*.

.

<img src="http://www.dlsi.ua.es/~jcalvo/curso-deep-learning/two-lstm.png" width="400">


Un ejemplo por código, usando el API secuencial.

In [ ]:
%%capture --no-stdout

T = 100
F = 2

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer((T, F)))
model.add(tf.keras.layers.LSTM(16,return_sequences=True))
model.add(tf.keras.layers.LSTM(16,return_sequences=True))
model.add(tf.keras.layers.LSTM(16,return_sequences=True))
model.add(tf.keras.layers.LSTM(10))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 100, 16)           1216      
                                                                 
 lstm_5 (LSTM)               (None, 100, 16)           2112      
                                                                 
 lstm_6 (LSTM)               (None, 100, 16)           2112      
                                                                 
 lstm_7 (LSTM)               (None, 10)                1080      
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
Total params: 6,630
Trainable params: 6,630
Non-trainable params: 0
_________________________________________________________________


#### Recurrencia bidireccional

En la mayoría de tareas, es interesante tener no sólo el contexto anterior sino también el posterior a la hora de predecir un elemento en un instante dado. En tal caso, podemos establecer una **recurrencia bidireccional**, en las cuales una neurona tiene dos conexiones recurrentes que, **a la hora de desplegarse**, lo hacen en direcciones opuestas.

![texto alternativo](https://cdn-images-1.medium.com/max/764/1*6QnPUSv_t9BY9Fv8_aLb-Q.png)

Para implementar redes bidireccionales Keras aporta un *wrapper* que acepta como parámetro un objeto que implemente la super clase recurrente:

* [keras.layers.Bidirectional](https://keras.io/api/layers/recurrent_layers/bidirectional/)

In [ ]:
T = 100
F = 2

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer((T, F)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 32)               2432      
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 10)                330       
                                                                 
Total params: 2,762
Trainable params: 2,762
Non-trainable params: 0
_________________________________________________________________


**Pregunta**: ¿Cuál es la operación que une las dos salidas individuales de una capa bidireccional?